In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

insurance_df = pd.read_csv("../Data/insurance.csv")

insurance_df.head()

In [0]:
# Feature Engineering

insurance_df = (
    pd.get_dummies(insurance_df, drop_first=True)
    .assign(
        age_sq = insurance_df["age"]**2,
        smoker_bmi_int = lambda x: x["smoker_yes"] * x["bmi"]
           )
)

In [0]:
# Data Splitting

from sklearn.model_selection import train_test_split

X = sm.add_constant(insurance_df.drop(["charges"], axis=1))
y = insurance_df["charges"]

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=2023)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.25, random_state=2023)

In [0]:
# Standardization

from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_tr = std.fit_transform(X_train.values)
X_val = std.transform(X_valid.values)
X_te = std.transform(X_test.values)

In [0]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=1).fit(X_tr, y_train)

list(zip(X.columns, ridge_model.coef_))

In [0]:
n_alphas = 200
alphas = 10 ** np.linspace(-3, 3, n_alphas)

train_scores = []
val_scores = []

for alpha in alphas:
    ridge_model = Ridge(alpha=alpha).fit(X_tr, y_train)
    train_scores.append(ridge_model.score(X_tr, y_train))
    val_scores.append(ridge_model.score(X_val, y_valid))
    
scores_df = pd.DataFrame({
    "alpha": alphas,
    "training": train_scores,
    "validation": val_scores
})

scores_df[:150].set_index("alpha").plot() #[:150]

In [0]:
scores_df.sort_values("validation", ascending=False)

In [0]:
std = StandardScaler()
X_m = std.fit_transform(X.values)
X_te = std.transform(X_test.values)

In [0]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=1.933).fit(X_m, y)

list(zip(X.columns, ridge_model.coef_))

In [0]:
ridge_model.score(X_m, y)

In [0]:
ridge_model.score(X_te, y_test)

In [0]:
lr_model = sm.OLS(y, X).fit()

# lr_model.summary()

In [0]:
from sklearn.metrics import r2_score as r2

r2(y_test, lr_model.predict(X_test))

# RidgeCV

In [0]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error as mae

n_alphas = 200
alphas = 10 ** np.linspace(-3, 3, n_alphas)

ridge_model = RidgeCV(alphas=alphas, cv=5)

ridge_model.fit(X_m, y)
print(ridge_model.score(X_m, y))
print(mae(y, ridge_model.predict(X_m)))
print(ridge_model.alpha_)

In [0]:
ridge_model.score(X_te, y_test)

## Lasso Regression

In [0]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_tr = std.fit_transform(X_train.values)
X_val = std.transform(X_valid.values)
X_te = std.transform(X_test.values)

In [0]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=50).fit(X_tr, y_train) # up strength to drop vars

list(zip(X.columns, lasso_model.coef_))

In [0]:
n_alphas = 200
alphas = 10 ** np.linspace(-3, 3, n_alphas)

train_scores = []
val_scores = []

for alpha in alphas:
    lasso_model = Lasso(alpha=alpha).fit(X_tr, y_train)
    train_scores.append(lasso_model.score(X_tr, y_train))
    val_scores.append(lasso_model.score(X_val, y_valid))
    
scores_df = pd.DataFrame({
    "alpha": alphas,
    "training": train_scores,
    "validation": val_scores
})

scores_df.set_index("alpha").plot() #[:150]

In [0]:
scores_df.sort_values("validation", ascending=False)

In [0]:
lasso_model = Lasso(alpha=31.08).fit(X_m, y)

list(zip(X.columns, lasso_model.coef_))

In [0]:
lasso_model.score(X_m, y)

In [0]:
lasso_model.score(X_te, y_test)

In [0]:
r2(y_test, lr_model.predict(X_test))

## LassoCV

In [0]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_absolute_error as mae

n_alphas = 200
alphas = 10 ** np.linspace(-3, 3, n_alphas)

lasso_model = LassoCV(alphas=alphas, cv=5)

lasso_model.fit(X_m, y)
print(lasso_model.score(X_m, y))
print(mae(y, lasso_model.predict(X_m)))
print(lasso_model.alpha_)

In [0]:
lasso_model.score(X_te, y_test)

In [0]:
list(zip(X.columns, lasso_model.coef_))

### Elastic Net

In [0]:
from sklearn.linear_model import ElasticNet

enet_model = ElasticNet(alpha=1, l1_ratio=.5).fit(X_tr, y_train)

print(
    f"Train Score: {round(enet_model.score(X_tr, y_train), 4)} "
    f"Valid Score: {round(enet_model.score(X_val, y_valid), 4)} "
)

In [0]:
train_scores = []
val_scores = []

l1_ratios = np.linspace(.01, 1, 100)

for l1_ratio in l1_ratios:
    enet_model = ElasticNet(alpha=1, l1_ratio=l1_ratio).fit(X_tr, y_train)
    train_scores.append(enet_model.score(X_tr, y_train))
    val_scores.append(enet_model.score(X_val, y_valid))

In [0]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(l1_ratios, train_scores, label="Train Scores")
ax.plot(l1_ratios, val_scores, label="Validation Scores")

ax.set_xlabel("L1_Ratio")
ax.set_ylabel("Prediction R2")

ax.spines[["top", "right"]].set_visible(False)
ax.legend()

In [0]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV


l1_ratios = np.linspace(.01, 1, 100)
alphas = 10 ** np.linspace(-3, 3, n_alphas)

param_grid = {"alpha": alphas, "l1_ratio": l1_ratios}

eNet_model = ElasticNet()
grid = GridSearchCV(eNet_model, param_grid, scoring='r2', cv=5)
grid.fit(X_m, y)

In [0]:
print(f"Best Params: {grid.best_params_}")
print(f"Best Score: {grid.best_score_}")

In [0]:
eNet_final = ElasticNet(alpha=1.11, l1_ratio=1).fit(X_m, y)

eNet_final.score(X_m, y)

In [0]:
eNet_final.score(X_te, y_test)

## Elastic Net CV

In [0]:
from sklearn.linear_model import ElasticNetCV

alphas = 10 ** np.linspace(-3, 3, 200)
l1_ratios = np.linspace(.9, 1, 10) # .9, 1, 10

enet_model = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratios, cv=5)

enet_model.fit(X_m, y)
print(enet_model.score(X_m, y))
print(enet_model.score(X_te, y_test))
print(enet_model.alpha_)
print(enet_model.l1_ratio_)

### Tree Based Models

In [0]:
import pandas as pd

insurance_df = pd.read_csv("../Data/insurance.csv")

insurance_df.head()

In [0]:
from sklearn.model_selection import train_test_split

insurance_df = pd.get_dummies(insurance_df, drop_first=True)

X = insurance_df.drop(["charges"], axis=1)
y = insurance_df["charges"]

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=2023)

In [0]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(max_depth=6)

model.fit(X, y)

print(model.score(X, y))
print(model.score(X_test, y_test))

In [0]:
X.head()

In [0]:
from sklearn.tree import plot_tree

plot_tree(model);

In [0]:
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(min_samples_leaf=5, n_estimators=1000, random_state=2023)

model = rf.fit(X, y)

print(model.score(X, y))
print(model.score(X_test, y_test))

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(min_samples_leaf=5, n_estimators=50, random_state=2023)

model = gb.fit(X, y)

print(model.score(X, y))
print(model.score(X_test, y_test))